In [1]:
import os

In [3]:
pwd

u'/Users/harishupadhyay/harish/ml/BuildingMachineLearningSystemsWithPython'

In [10]:
import utils

In [11]:
ls

analyze_webstats.py  gen_webstats.py      utils.pyc
charts/              performance_test.py
data/                utils.py


In [12]:
cd ch01

[Errno 2] No such file or directory: 'ch01'
/Users/harishupadhyay/harish/ml/BuildingMachineLearningSystemsWithPython/ch01


In [13]:
pwd

u'/Users/harishupadhyay/harish/ml/BuildingMachineLearningSystemsWithPython/ch01'

In [14]:
from utils import DATA_DIR,CHART_DIR

In [16]:
import scipy as sp

In [17]:
import matplotlib.pyplot as plt 

In [18]:
sp.random.seed(3)

In [20]:
data = sp.genfromtxt(os.path.join(DATA_DIR,"web_traffic.tsv"),delimiter="\t")

In [21]:
print (data[:10])

[[  1.00000000e+00   2.27200000e+03]
 [  2.00000000e+00              nan]
 [  3.00000000e+00   1.38600000e+03]
 [  4.00000000e+00   1.36500000e+03]
 [  5.00000000e+00   1.48800000e+03]
 [  6.00000000e+00   1.33700000e+03]
 [  7.00000000e+00   1.88300000e+03]
 [  8.00000000e+00   2.28300000e+03]
 [  9.00000000e+00   1.33500000e+03]
 [  1.00000000e+01   1.02500000e+03]]


In [22]:
print (data.shape)

(743, 2)


In [23]:
colors = ['g','k','b','m','r']

In [24]:
linestyles = ['-','-.','--',':','-']

In [25]:
x = data[:,0]

In [26]:
y = data[:,1]

In [27]:
print("Number of invalid entries",sp.sum(sp.isnan(y)))

('Number of invalid entries', 8)


In [28]:
x= x[~sp.isnan(y)]

In [29]:
y = y[~sp.isnan(y)]

In [40]:
def plot_models(x,y,models,fname,mx=None,ymax=None,xmin=None):
    '''plot input data'''
    plt.figure(num=None,figsize=(8,6))
    plt.clf()
    plt.scatter(x,y,s=10)
    plt.title("web traffic over the last month")
    plt.xlabel("Time")
    plt.ylabel("Hits/hour")
    plt.xticks([w*7*24 for w in range(10)],['week %i'%w for w in range(10)])
    if models:
        if mx is None:
            mx = sp.linspace(0,x[-1],1000)
        for model,style, color in zip(models,linestyles,colors):
            plt.plot(mx,model(mx),linestyle=style,linewidth=2,c=color)
        plt.legend(["d=%i"%m.order for m in models],loc="upper left")
    plt.autoscale(tight=True)
    plt.ylim(ymin=0)
    if ymax:
        plt.xlim(xmin=xmin)
    plt.grid(True,linestyle='-',color='0.75')
    plt.savefig(fname)

#look at data
plot_models(x,y,None , os.path.join(CHART_DIR,"1400_01_01.png"))

In [41]:
# create and plot models
fp1, res1, rank1 , sv1, rcondl = sp.polyfit(x,y,1,full=True)

In [42]:
print("Model parameters of fp1: %s"%fp1)

Model parameters of fp1: [   2.59619213  989.02487106]


In [43]:
print("Error of the model of fp1:",res1)

('Error of the model of fp1:', array([  3.17389767e+08]))


In [44]:
fp2, res2, rank2 , sv2, rcond2 = sp.polyfit(x,y,2,full=True)

In [45]:
print("Model parameters of fp2 : %s"%fp2)

Model parameters of fp2 : [  1.05322215e-02  -5.26545650e+00   1.97476082e+03]


In [46]:
print("Error of models fp2:%s",fp2)

('Error of models fp2:%s', array([  1.05322215e-02,  -5.26545650e+00,   1.97476082e+03]))


In [53]:
f1 = sp.poly1d(fp1)

In [54]:
fp2,res2,rank2,sv2,rcond2=sp.polyfit(x,y,2,full=True)

In [55]:
print("Model parameters of fp2:%s"%fp2)

Model parameters of fp2:[  1.05322215e-02  -5.26545650e+00   1.97476082e+03]


In [56]:
print("Error of the model of fp2:",res2)

('Error of the model of fp2:', array([  1.79983508e+08]))


In [59]:
f2=sp.poly1d(fp2)
f3 =sp.poly1d(sp.polyfit(x,y,3))

In [60]:
f10=sp.poly1d(sp.polyfit(x,y,10))

In [61]:
f100=sp.poly1d(sp.polyfit(x,y,100))

/Users/harishupadhyay/anaconda/lib/python2.7/site-packages/numpy/lib/polynomial.py:583: RuntimeWarning: overflow encountered in multiply
  scale = NX.sqrt((lhs*lhs).sum(axis=0))
/Users/harishupadhyay/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: RankWarning: Polyfit may be poorly conditioned
  """Entry point for launching an IPython kernel.


In [62]:
plot_models(x,y,[f1],os.path.join(CHART_DIR,"1400_01_02.png"))

In [63]:
plot_models(x,y,[f1,f2],os.path.join(CHART_DIR,"1400_01_03.png"))

In [64]:
plot_models(x,y,[f1,f2,f3,f10,f100],os.path.join(CHART_DIR,"1400_01_04.png"))

In [70]:
inflection = int(3.5 * 7* 24)
print inflection

588


In [71]:
xa = x[:inflection]

In [72]:
ya = y[:inflection]

In [73]:
xb = x[inflection:]

In [74]:
yb = y [inflection:]

In [76]:
fa = sp.poly1d(sp.polyfit(xa,ya,1))

In [77]:
fb = sp.poly1d(sp.polyfit(xb,yb,1))

In [78]:
plot_models(x,y,[fa,fb],os.path.join(CHART_DIR,"1400_01_05.png"))

In [79]:
def error(f,x,y):
    return sp.sum((f(x)-y)*2)

In [80]:
print("Errors for the completed data set:")
for f in [f1,f2,f3,f10,f100]:
    print("Eroor d=%i:%f"%(f.order,error(f,xb,yb)))

Errors for the completed data set:
Eroor d=1:-138318.762585
Eroor d=2:-1517.439826
Eroor d=3:28328.495669
Eroor d=10:7857.513589
Eroor d=53:1933.099808


In [86]:
for f in [f1,f2,f3,f10,f100]:
    print("Error d=%i:%f"%(f.order,error(f,xb,yb)))

Error d=1:-138318.762585
Error d=2:-1517.439826
Error d=3:28328.495669
Error d=10:7857.513589
Error d=53:1933.099808


In [87]:
print("Error inflection=%f"%(error(fa,xa,ya)+error(fb,xb,yb)))

Error inflection=0.000000


In [90]:
plot_models(x,y,[f1,f2,f3,f10,f100],os.path.join(CHART_DIR,"1400_01_06.png"),
           mx=sp.linspace(0*7*24,6*7*24,100),
           ymax=10000,xmin=0*7*24)

In [91]:
print("Errors for only the time after inflection point")

Errors for only the time after inflection point


In [97]:
fb1 = fb
fb2 =sp.poly1d(sp.polyfit(xb,yb,2))
fb3 = sp.poly1d(sp.polyfit(xb,yb,3))
fb10 = sp.poly1d(sp.polyfit(xb,yb,10))
fb100 = sp.poly1d(sp.polyfit(xb,yb,100))
for f in [fb1,fb2,fb3,fb10,fb100]:
    print("Error d=%i:%f"%(f.order,error(f,xb,yb)))

Error d=1:-0.000000
Error d=2:-0.000000
Error d=3:-0.000000
Error d=10:0.004120
Error d=53:-2.478516


/Users/harishupadhyay/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: RankWarning: Polyfit may be poorly conditioned
  after removing the cwd from sys.path.
/Users/harishupadhyay/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: RankWarning: Polyfit may be poorly conditioned
  """


In [98]:
plot_models(x,y,[fb1,fb2,fb3,fb10,fb100],
           os.path.join(CHART_DIR,"1400_01_07.png"),
           mx=sp.linspace(0*7*24,6*7*24,100),
           ymax=10000,xmin=0*7*24)

In [112]:
frac = 0.3
split_idx = int(frac*len(xb))
shuffled = sp.random.permutation(list(range(len(xb))))
test = sorted(shuffled[:split_idx])
train = sorted(shuffled[split_idx:])
fbt1=sp.poly1d(sp.polyfit(xb[train],yb[train],3))
fbt2 = sp.poly1d(sp.polyfit(xb[train],yb[train],2))
print("fbt2(x)=\n%s"%fbt2)
print("fbt2(x)-100,000=\n%s"%(fbt2-100000))
fbt3=sp.poly1d(sp.polyfit(xb[train],yb[train],3))
fbt10=sp.poly1d(sp.polyfit(xb[train],yb[train],10))
fbt100 = sp.poly1d(sp.polyfit(xb[train],yb[train],100))

fbt2(x)=
         2
0.05155 x - 48.18 x + 1.223e+04
fbt2(x)-100,000=
         2
0.05155 x - 48.18 x - 8.777e+04


/Users/harishupadhyay/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:11: RankWarning: Polyfit may be poorly conditioned
  # This is added back by InteractiveShellApp.init_path()
/Users/harishupadhyay/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:12: RankWarning: Polyfit may be poorly conditioned
  if sys.path[0] == '':


In [113]:
print("Test errors for only the time after inflection point")

Test errors for only the time after inflection point


In [114]:
for f in [fbt1,fbt2,fbt3,fbt10,fbt100]:
    print("Error d=%i:%f"%(f.order,error(f,xb[test],yb[test])))

Error d=3:-5990.308316
Error d=2:-5989.075311
Error d=3:-5990.308316
Error d=10:-9837.516113
Error d=53:-9094.468750


In [116]:
plot_models(x,y,[fbt1,fbt2,fbt3,fbt10,fbt100],os.path.join(CHART_DIR,"1400_01_08.png"),
           mx=sp.linspace(0*7*24,6*7*24,100),
           ymax=10000,xmin=0*7*24)

In [118]:
from scipy.optimize import fsolve

In [119]:
print(fbt2)

         2
0.05155 x - 48.18 x + 1.223e+04


In [120]:
print(fbt2-100000)

         2
0.05155 x - 48.18 x - 8.777e+04


In [121]:
reached_max=fsolve(fbt2-100000,x0=800)/(7*24)

In [123]:
print("100,1000 hits/hour expected at week %f " %reached_max[0])

100,1000 hits/hour expected at week 11.031938 
